In [27]:
import numpy as np
import pandas as pd
import math
import re
import sys

np.set_printoptions(threshold=sys.maxsize)

In [28]:
FILE_NAME = "test.txt"
#FILE_NAME = "input.txt"

DIAG=False
DIAG2 = True
TARGET = 1000000000

Alternative approach seen was to sort map within the bound designated by rock. Would have to represent columns as arrays 
or use slicing on pandas or numpy arrays with some smart substitution

In [29]:
def buildArrayFromLines(lines):
    
    tmp_arr = []
    for line in lines:
        tmp_line = []    
        
        for c in line:            
            if c == ".":
                tmp_line += [0]
            elif c == "#":
                tmp_line += [-1]
            elif c == "O":
                tmp_line += [1]
            elif c == "\n":
                pass
            else:
                print("THIS SHOULDN'T HAVE HAPPENED: ",print(c),"-",line,"-")
        tmp_arr.append(tmp_line)
    
    
    return np.array(tmp_arr)
    

In [30]:
def tiltMap_OneColumn(my_map, col_num,row_num=0):
    res = my_map
        
    shape_X,_ = my_map.shape
    
    if row_num >= shape_X:
        return res
    
    if my_map[row_num,col_num] in [-1,1]: # move on
        return tiltMap_OneColumn(my_map,col_num,row_num+1)
    else: # looks like we have empty space
    
        j = row_num+1              
        while j < shape_X and my_map[j,col_num] == 0:
            j+=1
        
        if j==shape_X:
            return res
        elif my_map[j,col_num] == -1: # nothing moves past the rock
            return tiltMap_OneColumn(my_map,col_num,j+1)
        elif my_map[j,col_num] == 1:
            my_map[row_num,col_num]=1
            my_map[j,col_num]=0
            return tiltMap_OneColumn(my_map,col_num,row_num+1)
        else:
            print("This really shouldn't have happened!")
                    
    return res
    

In [31]:
def tiltMap(my_map):
    res = my_map
    shape_X,shape_Y = my_map.shape
    
    for i in range(0,shape_Y):
        res = tiltMap_OneColumn(res,i)
    return res

In [32]:
def shuffleMap(my_map,direction,reverse=False):
    res = my_map.copy()
    
    if direction =="N" :
        pass
    elif direction =="S" :
        res = res[::-1]
    elif direction =="W" :
        res = np.transpose(res)
    elif direction =="E" :
        if reverse:            
            res = res[::-1]
            res = np.transpose(res)
        else:
            res = np.transpose(res)
            res = res[::-1]
    else:
        print("Inadmissible inputs!!!")
    return res
        


In [33]:
def tiltMap_fullCycle(my_map):
    res = my_map.copy()
    if DIAG: print("----")
    if DIAG: print(res)
    for direction in ["N","W","S","E"]:
        res = shuffleMap(res,direction,False)
        res = tiltMap(res)        
        res = shuffleMap(res,direction,True)
        
        if DIAG: print("----")
        if DIAG: print(direction)
        if DIAG: print(res)
    return res
    

In [34]:
# OK
def parseFile(fileName):   
    with open(fileName) as file:
        lines =  file.readlines()    
   
    my_arr = buildArrayFromLines(lines)
    
    
    return my_arr


In [35]:
def tallyUpMap(my_map):
    res = 0
    tmp_map = my_map.copy()
    tmp_map[tmp_map==-1]=0
    
    shape_X,shape_Y = my_map.shape
    for i in range(0,shape_X):
        tmp_val= sum(tmp_map[i])*(shape_X-i)        
        res += tmp_val
    
    return res
        

In [36]:
def loadingAfterNorthTilt(map_to_tilt):
    x=tiltMap(map_to_tilt)
    return tallyUpMap(x)

In [37]:
def loadingAfter4SOWTilts(map_to_tilt,numCycles):
    res_map = []
    res_score = []

    for counter in range(0,numCycles):        
        map_to_tilt=tiltMap_fullCycle(map_to_tilt)
        if str(map_to_tilt) in res_map:
            cycle_start = res_map.index(str(map_to_tilt))
            if DIAG: print("pop",cycle_start,counter)
            break
        else:
            res_map.append(str(map_to_tilt))
            res_score.append(tallyUpMap(map_to_tilt))

  
    cycle_length = len(res_map)-cycle_start
    res_idx = ((TARGET-1-cycle_start) % cycle_length)+cycle_start

    return res_score[res_idx]


In [38]:


mirror_map = parseFile(FILE_NAME)

resA = loadingAfterNorthTilt(mirror_map.copy())
resB = loadingAfter4SOWTilts(mirror_map.copy(),TARGET)
print("Result part A:",resA)
print("Result part B:",resB)



Result part A: 109466
Result part B: 94585


# Interesting
Note: Alternatively could have hashed numpy arrays, using e.g. sha1

In [39]:
from hashlib import sha1

mirror_map_ro = mirror_map.copy()
mirror_map_ro.flags.writeable = False
foo = sha1(mirror_map_ro.data)
print(foo.hexdigest())

64fce98f6764b6bcb568f26bffed920ca6dc2c84
